In [ ]:
import gradio as gr
import cv2
import numpy as np
import time
from PIL import Image as PILImage
from ultralytics import YOLO
from transformers import CLIPProcessor, CLIPModel
from gtts import gTTS 
import random

gr.close_all()

class GymkhanaMaster:
    def __init__(self):
        print("🛠️ System Init: Cargando Modelos Neurales...")
        # Modelos: YOLO para detectar (rápido) y CLIP para comparar (inteligente)
        self.yolo = YOLO('yolov8n.pt')
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        
        # Variables de estado y puntuación (NUEVO)
        self.current_target = None
        self.time_limit = 60
        self.start_time = 0
        self.game_active = False
        self.difficulty_multiplier = 1.0
        self.score = 0 

    def _hablar(self, texto):
        """Genera el audio MP3 para la voz del juego"""
        try:
            tts = gTTS(text=texto, lang='es')
            filename = "audio_mision.mp3"
            tts.save(filename)
            return filename
        except Exception:
            return None

    def configurar_partida(self, dificultad):
        """Configura tiempo y multiplicador de puntos según dificultad"""
        config = {
            "🟢 Fácil (60s)": (60, 1.0), 
            "🟡 Medio (30s)": (30, 2.0), 
            "🔴 Difícil (15s)": (15, 5.0)
        }
        self.time_limit, self.difficulty_multiplier = config.get(dificultad, (60, 1.0))
        return self.time_limit

    def generar_narrativa(self, objeto):
        """Crea el HTML de la tarjeta de misión y el texto para la voz"""
        obj_upper = objeto.upper()
        html_visual = f"""
        <div class="mission-card">
            <div class="mission-icon">🎯 OBJETIVO</div>
            <div class="mission-target">{obj_upper}</div>
            <div class="mission-desc">ENCUENTRA ESTE OBJETO RÁPIDO</div>
        </div>
        """
        voz = f"Misión iniciada. Encuentra el objeto: {objeto}. Tienes {self.time_limit} segundos."
        return html_visual, voz

    def estampar_sello(self, imagen_numpy, texto_sello, es_victoria=True):
        """Dibuja el marco de neón y el sello de resultado sobre la imagen final"""
        img = imagen_numpy.copy()
        h, w = img.shape[:2]
        color = (57, 255, 20) if es_victoria else (255, 0, 0) 
        
        # Marco exterior y Texto centrado
        cv2.rectangle(img, (0,0), (w, h), color, 20)
        texto_sello = texto_sello.upper()
        font = cv2.FONT_HERSHEY_DUPLEX
        scale = 1.5 if w < 400 else 3.0
        thickness = 3 if w < 400 else 6
        (tw, th), _ = cv2.getTextSize(texto_sello, font, scale, thickness)
        cx, cy = w // 2, h // 2
        cv2.rectangle(img, (cx - tw//2 - 20, cy - th - 20), (cx + tw//2 + 20, cy + 20), (0,0,0), -1)
        cv2.putText(img, texto_sello, (cx - tw//2, cy), font, scale, color, thickness)
        return img

    def escanear_sala(self, imagen_sala):
        """Paso 1: Analiza la foto, elige objeto y crea el recorte (Target Lock)"""
        if imagen_sala is None: return '<div class="status-msg">📸 SUBE UNA FOTO</div>', None, None, None
        
        pil_img = PILImage.fromarray(imagen_sala)
        cv_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        res = self.yolo(cv_img, conf=0.25, verbose=False)[0]
        
        # Extraemos coordenadas para poder recortar luego
        detection_data = []
        for box in res.boxes:
            name = self.yolo.names[int(box.cls[0])]
            coords = box.xyxy[0].cpu().numpy().astype(int)
            detection_data.append({"name": name, "coords": coords})
        
        if not detection_data: 
            return '<div class="status-error">⚠️ NO VEO NADA CLARO</div>', None, None, None

        # Selección aleatoria
        seleccion = random.choice(detection_data)
        self.current_target = seleccion["name"]
        
        # --- CREAR RECORTE ---
        x1, y1, x2, y2 = seleccion["coords"]
        imagen_recorte = imagen_sala[y1:y2, x1:x2]
        
        self.start_time = time.time()
        self.game_active = True
        
        html_mision, txt_voz = self.generar_narrativa(self.current_target)
        ruta_audio = self._hablar(txt_voz)
        
        html_barra = f"""
        <div class="timer-wrapper">
            <div class="timer-fill" style="animation-duration: {self.time_limit}s;"></div>
        </div>
        <div class="timer-label">⏳ TIEMPO RESTANTE</div>
        """
        return html_mision, ruta_audio, html_barra, imagen_recorte

    def verificar_jugada(self, imagen_usuario):
        """Paso 2: Verifica con la cámara y genera el Radar de Proximidad"""
        if not self.game_active: return "error", "STOP", "Juego no iniciado", None, None
        if imagen_usuario is None: return "error", "CÁMARA", "No hay imagen", None, None

        pil_img = PILImage.fromarray(imagen_usuario)
        opciones = [f"a photo of a {self.current_target}", "something else"]
        inputs = self.clip_processor(text=opciones, images=pil_img, return_tensors="pt", padding=True)
        probs = self.clip_model(**inputs).logits_per_image.softmax(dim=1)
        
        # Obtenemos confianza numérica para el Radar
        score_confianza = probs[0][0].item()
        
        # --- GENERAR HTML DEL RADAR ---
        porcentaje = int(score_confianza * 100)
        color_radar = "#ff0000" # Rojo
        if porcentaje > 40: color_radar = "#ffff00" # Amarillo
        if porcentaje > 60: color_radar = "#39ff14" # Verde Neon
        
        html_radar = f"""
        <div style="background:#333; border-radius:10px; padding:5px; margin-top:10px; border: 2px solid #555;">
            <div style="color:white; font-size:12px; margin-bottom:2px;">DETECTANDO SIMILITUD: {porcentaje}%</div>
            <div style="width:100%; height:15px; background:#000; border-radius:5px;">
                <div style="width:{porcentaje}%; height:100%; background:{color_radar}; border-radius:5px; transition: width 0.3s;"></div>
            </div>
        </div>
        """

        # LÓGICA DE VICTORIA Y PUNTOS
        if probs.argmax().item() == 0 and score_confianza > 0.6:
            self.game_active = False
            
            # Cálculo de puntos (tiempo sobrante * dificultad)
            tiempo_usado = time.time() - self.start_time
            tiempo_restante = max(0, self.time_limit - tiempo_usado)
            puntos_ronda = int(tiempo_restante * 100 * self.difficulty_multiplier)
            self.score += puntos_ronda
            
            self._hablar(f"Correcto. Has ganado {puntos_ronda} puntos.")
            img_final = self.estampar_sello(imagen_usuario, f"+{puntos_ronda} PTS", True)
            
            return "win", "¡MISIÓN CUMPLIDA!", f"Objeto: {self.current_target} | Puntos Ronda: {puntos_ronda}", img_final, html_radar
        
        # Pista intermedia
        elif probs.argmax().item() == 0 and score_confianza > 0.35:
            self._hablar("Lo veo, pero acércalo más.")
            return "retry", "⚠️ ACÉRCALO MÁS", "Detecto el objeto, pero está lejos o borroso.", None, html_radar
            
        else:
            self._hablar("Objeto incorrecto.")
            return "retry", "❌ ERROR DE ESCANEO", "Eso no parece el objeto...", None, html_radar

    def check_tiempo(self):
        """Revisa si se acabó el tiempo y penaliza puntos"""
        if self.game_active and (time.time() - self.start_time > self.time_limit):
            self.game_active = False
            self.score = max(0, self.score - 500) # Penalización
            self._hablar("Tiempo agotado. Pierdes puntos.")
            return True
        return False
    
    def get_score_html(self):
        return f'<div class="score-board">🏆 SCORE: {self.score:05d}</div>'

juego = GymkhanaMaster()


css_arcade = """
@import url('https://fonts.googleapis.com/css2?family=Press+Start+2P&family=Roboto:wght@400;900&display=swap');

body { background-color: #121212; color: #ffffff; font-family: 'Roboto', sans-serif; }
.gradio-container { max-width: 900px !important; margin: auto; }

/* Títulos y Marcador */
.main-title { font-family: 'Press Start 2P', cursive; color: #ff00ff; text-align: center; text-shadow: 4px 4px #00ffff; margin-bottom: 10px; }
.score-board { font-family: 'Press Start 2P', cursive; color: #39ff14; font-size: 1.5em; text-align: right; text-shadow: 0 0 10px #39ff14; margin-bottom: 10px; }

/* Paneles y Tarjetas */
.game-panel { background: #1e1e1e; border: 3px solid #333; border-radius: 10px; padding: 20px; margin-bottom: 20px; box-shadow: 0 10px 20px rgba(0,0,0,0.5); }
.mission-card { background: linear-gradient(135deg, #2b002b 0%, #3a0ca3 100%); border: 4px solid #00ffff; border-radius: 15px; padding: 20px; text-align: center; box-shadow: 0 0 20px #00ffff; animation: pulse 2s infinite; }
@keyframes pulse { 0% { box-shadow: 0 0 15px #00ffff; } 50% { box-shadow: 0 0 30px #00ffff; } 100% { box-shadow: 0 0 15px #00ffff; } }

/* Barra de Tiempo */
.timer-wrapper { width: 100%; height: 40px; background: #ffffff; border: 4px solid #ff0000; border-radius: 20px; overflow: hidden; margin-top: 15px; }
.timer-fill { height: 100%; background: #ff0000; width: 0%; animation-name: countdown; animation-timing-function: linear; animation-fill-mode: forwards; }
@keyframes countdown { from { width: 100%; } to { width: 0%; } }

/* Corrección de Imágenes para que no se corten */
.img-ajustada { display: flex !important; justify-content: center !important; align-items: center !important; background-color: transparent !important; border: none !important; height: auto !important; padding: 0 !important; }
.img-ajustada img { object-fit: contain !important; width: 100% !important; height: auto !important; max-height: 80vh !important; display: block !important; margin: 0 auto !important; }

/* Estilos de botones (Start, Scan, Action) omitidos por brevedad pero incluidos en el código completo anterior */
"""


with gr.Blocks(theme=gr.themes.Base(), css=css_arcade) as demo:
    
    timer = gr.Timer(1, active=False)
    
    # --- PANTALLA 1: MENÚ ---
    with gr.Column(visible=True, elem_classes=["game-panel"]) as p_menu:
        gr.Markdown("# 🕹️ OBGUESSER ARCADE", elem_classes=["main-title"])
        gr.Markdown("ENCUENTRA EL OBJETO REAL ANTES DE QUE ACABE EL TIEMPO", elem_classes=["sub-title"])
        html_score_menu = gr.HTML(value=juego.get_score_html()) # Marcador Global
        sel_dif = gr.Radio(["🟢 Fácil (60s)", "🟡 Medio (30s)", "🔴 Difícil (15s)"], label="NIVEL DE DIFICULTAD", value="🟢 Fácil (60s)")
        btn_start = gr.Button("¡EMPEZAR PARTIDA!", elem_classes=["btn-start"])

    # --- PANTALLA 2: JUEGO ---
    with gr.Column(visible=False) as p_juego:
        with gr.Row():
            with gr.Column(scale=2): html_obj = gr.HTML(value='<div class="status-msg">ESPERANDO ANÁLISIS...</div>')
            with gr.Column(scale=1): 
                gr.Markdown("**🎯 BUSCA ESTO:**")
                img_crop = gr.Image(show_label=False, interactive=False, height=100) # El recorte
            with gr.Column(scale=1): html_bar = gr.HTML()

        with gr.Row():
            with gr.Column(elem_classes=["game-panel"]):
                gr.Markdown("### 1. SUBE FOTO DE TU HABITACIÓN")
                # Soporta Upload y Webcam
                in_sala = gr.Image(sources=["upload", "webcam"], type="numpy", height=200, show_label=False)
                btn_scan = gr.Button("🔍 ANALIZAR OBJETOS", elem_classes=["btn-scan"])
            
            with gr.Column(elem_classes=["game-panel"]):
                gr.Markdown("### 2. ENSEÑA EL OBJETO AQUI")
                in_cam = gr.Image(sources=["webcam"], type="numpy", height=200, show_label=False)
                html_radar = gr.HTML() # El radar de proximidad
                btn_ok = gr.Button("✅ ¡LO TENGO!", elem_classes=["btn-action"])
        
        html_fb = gr.HTML()
        aud_out = gr.Audio(autoplay=True, visible=False)

    # --- PANTALLA 3: FINAL ---
    with gr.Column(visible=False, elem_classes=["game-panel"]) as p_final:
        gr.Markdown("# RESULTADO FINAL", elem_classes=["main-title"])
        html_score_final = gr.HTML()
        img_result = gr.Image(show_label=False, interactive=False, elem_classes=["img-ajustada"])
        lbl_res_titulo = gr.Markdown(elem_classes=["sub-title"])
        lbl_res_desc = gr.Markdown(elem_classes=["status-msg"])
        btn_reset = gr.Button("🔄 JUGAR OTRA VEZ", elem_classes=["btn-start"])


    
    # Inicio: Resetea variables visuales y asegura que el botón Scan se vea
    def start_game(d):
        juego.configurar_partida(d)
        return {p_menu: gr.Column(visible=False), p_juego: gr.Column(visible=True), p_final: gr.Column(visible=False), 
                in_sala: None, in_cam: None, html_obj: '<div class="status-msg">📸 SUBE UNA FOTO</div>', 
                html_bar: "", img_crop: None, html_radar: "", btn_scan: gr.update(visible=True)}

    # Escaneo: Genera misión, audio, barra tiempo, recorte y OCULTA el botón scan
    def scan_room(img):
        h_mision, ruta_audio, h_bar, recorte = juego.escanear_sala(img)
        act_btn = gr.update(visible=False) if h_bar else gr.update(visible=True)
        return h_mision, ruta_audio, h_bar, recorte, gr.Timer(active=bool(h_bar)), act_btn

    # Verificación: Comprueba victoria y actualiza Radar
    def verify_obj(img):
        res, tit, msg, final_img, radar_html = juego.verificar_jugada(img)
        if res == "retry": return {html_fb: f'<div class="status-error">{msg}</div>', html_radar: radar_html}
        return {p_juego: gr.Column(visible=False), p_final: gr.Column(visible=True), img_result: final_img, 
                lbl_res_titulo: f"## {tit}", lbl_res_desc: msg, timer: gr.Timer(active=False), 
                html_score_final: juego.get_score_html()}

    # Timer: Checkea tiempo y penaliza score
    def time_check():
        if juego.check_tiempo():
            black = np.zeros((400,600,3), dtype=np.uint8)
            fail = juego.estampar_sello(black, "TIEMPO FUERA", False)
            return (gr.update(visible=False), gr.update(visible=True), fail, "## ⌛ TIEMPO AGOTADO", gr.Timer(active=False), juego.get_score_html())
        return (gr.update(), gr.update(), gr.update(), gr.update(), gr.update(), gr.update())

    # Conexiones con show_progress="hidden" para limpieza visual
    btn_start.click(start_game, [sel_dif], [p_menu, p_juego, p_final, in_sala, in_cam, html_obj, html_bar, img_crop, html_radar, btn_scan], show_progress="hidden")
    btn_scan.click(scan_room, in_sala, [html_obj, aud_out, html_bar, img_crop, timer, btn_scan], show_progress="hidden")
    btn_ok.click(verify_obj, in_cam, [html_fb, p_juego, p_final, img_result, lbl_res_titulo, lbl_res_desc, timer, html_score_final, html_radar], show_progress="hidden")
    timer.tick(time_check, outputs=[p_juego, p_final, img_result, lbl_res_titulo, timer, html_score_final], show_progress="hidden")
    btn_reset.click(lambda: {p_menu: gr.Column(visible=True), p_final: gr.Column(visible=False)}, outputs=[p_menu, p_final], show_progress="hidden").then(lambda: juego.get_score_html(), outputs=html_score_menu)

demo.launch()

Closing server running on port: 7860
🛠️ System Init: Cargando Modelos Neurales...


C:\Users\Nitropc\AppData\Local\Temp\ipykernel_18068\3720382155.py:246: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Base(), css=css_arcade) as demo:


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Nitropc\anaconda3\envs\IPM2\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nitropc\anaconda3\envs\IPM2\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nitropc\anaconda3\envs\IPM2\Lib\site-packages\fastapi\applications.py", line 1135, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Nitropc\anaconda3\envs\IPM2\Lib\site-packages\starlette\applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Nitropc\anaconda3\envs\IPM2\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  Fil